In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import random

In [ ]:
random.seed(5566)
num_lines = 40428968
sample_size = 2000000
skip = sorted(random.sample(range(1, num_lines), num_lines-sample_size))
skip[0]

In [ ]:
df= pd.read_csv("../input/avazu-ctr-prediction/train.gz", header=0, skiprows=skip)

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df["hour"] = pd.to_datetime(df["hour"], format="%y%m%d%H")

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
sns.countplot(x='click',data=df)


Here we can see that the number of clicks is very less as compared to the number of non clicks

In [ ]:
df.dtypes

# CTR current

In [ ]:
df["click"].value_counts() / len(df)

In [ ]:
df["hour"].dt.hour

In [ ]:
df["actual_hour_day"]= df["hour"].dt.hour

In [ ]:
df.head(2)

In [ ]:
imp= df.groupby(['actual_hour_day', 'click']).size().unstack()

In [ ]:
imp.head()

In [ ]:
plt.figure()
imp.plot(kind='bar', title="Day hours", figsize=(16,8))
plt.ylabel("count of clicks")
plt.plot()

Here we can see the hourly impressions/day and we can clearly see that the rate of clicks is very low as compared to non  clicks

In [ ]:
df.groupby('banner_pos')['click'].sum()

In [ ]:
df.groupby('device_type')['click'].sum()

In [ ]:
df.banner_pos.unique()

# CTR with respect to different columns

for C1

In [ ]:
df.columns

In [ ]:
df.C1.value_counts()/len(df)

In [ ]:
c1= df.C1.unique()
print(c1)

In [ ]:
ctr_list=[]
for i in c1:
    ctr=df.loc[np.where((df.C1 == i))].click.value_counts() / len(df)
    ctr_list.append(ctr)
    print("C1 values {}:, CTR is {}:".format(i,ctr))

FOR banner_pos

In [ ]:
df.banner_pos.value_counts() / len(df)

In [ ]:
banner= df.banner_pos.unique()
print(banner)

In [ ]:
ban_list=[]
for i in banner:
    baa=df.loc[np.where((df.banner_pos == i))].click.value_counts() / len(df)
    ban_list.append(baa)
    print("banner values {}:, banner is {}:".format(i,baa))

In [ ]:
df.dtypes

for device_type

In [ ]:
df.device_type.value_counts() / len(df)

In [ ]:
dev= df.device_type.unique()
print(dev)

In [ ]:
X= df.drop("click", axis=1)

y= df.click

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.3, random_state=12)

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_features= ["site_id", "site_domain", "site_category",
              "app_id", "app_domain", "app_category", "device_id", 
              "device_ip", "device_model"]

In [ ]:
cat_boost= CatBoostClassifier(learning_rate=0.05, loss_function='Logloss', 
                             random_state=123, iterations=100, eval_metric="Logloss")

In [ ]:
cat_boost.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features= cat_features, plot=True)

In [ ]:
c= cat_boost.get_feature_importance(prettified=True).sort_index(ascending=True)


In [ ]:
c.to_csv("feature_imp.csv")

In [ ]:
predictions= cat_boost.predict(X_test, prediction_type="Class", ntree_start=0, ntree_end= cat_boost.get_best_iteration(),thread_count=-1, 
                               verbose=None)

In [ ]:
predictions
